In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
import sys
#PATH = '/content/drive/My Drive/PPM_Stability/'
PATH = "C:/Users/velmurug/Documents/Stability Experiments/benchmark_interpretability/PPM_Stability/"
sys.path.append(PATH)

In [2]:
#!pip install lime
#!pip install shap
#!pip install pandas==0.19.2
!pip install xgboost==1.0.0

In [3]:
import EncoderFactory
from DatasetManager import DatasetManager
import BucketFactory
import stability as st #Nogueira, Sechidis, Brown.

import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler

import time
import os
import sys
from sys import argv
import pickle
from collections import defaultdict
import random
import joblib

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import lime
import lime.lime_tabular
from lime import submodular_pick;

import shap

import warnings
warnings.filterwarnings('ignore')

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

def imp_df(column_names, importances):
        df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
        return df

# plotting a feature importance dataframe (horizontal barchart)
def var_imp_plot(imp_df, title, num_feat):
        imp_df.columns = ['feature', 'feature_importance']
        b= sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df.head(num_feat), orient = 'h', palette="Blues_r")

In [5]:
def generate_global_explanations(train_X,train_Y, cls, feature_combiner):
    
    print("The number of testing instances is ",len(train_Y))
    print("The total number of columns is",train_X.shape[1]);
    print("The total accuracy is ",cls.score(train_X,train_Y));
       
    sns.set(rc={'figure.figsize':(10,10), "font.size":18,"axes.titlesize":18,"axes.labelsize":18})
    sns.set
    feat_names = feature_combiner.get_feature_names()
    base_imp = imp_df(feat_names, cls.feature_importances_)
    base_imp.head(15)
    var_imp_plot(base_imp, 'Feature importance using XGBoost', 15)
    return base_imp

In [6]:
from lime import submodular_pick
def generate_lime_explanations(explainer,test_xi, cls,test_y, submod=False, test_all_data=None):
    
    #print("Actual value ", test_y)
    num_features=max_feat;
    exp = explainer.explain_instance(test_xi, 
                                 cls.predict_proba, num_features=num_features, labels=[0,1])
    
    return exp
        
    if submod==True:
        sp_obj=submodular_pick.SubmodularPick(explainer, test_all_data, cls.predict_proba, 
                                      sample_size=20, num_features=num_features,num_exps_desired=4)
        [exp.as_pyplot_figure(label=exp.available_labels()[0]) for exp in sp_obj.sp_explanations];

In [7]:
def create_samples(shap_explainer, iterations, row, features, top = 10):
    length = len(features)
    
    exp = []
    rel_exp = []
    
    for j in range(iterations):
        #print(X_test_frame.loc[row])
        shap_values = shap_explainer.shap_values(row)
        #print(shap_values)

        importances = []
        
        if type(shap_explainer) == shap.explainers.kernel.KernelExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][i]
                abs_val = abs(shap_values[0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
                
        elif type(shap_explainer) == shap.explainers.tree.TreeExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][i]
                abs_val = abs(shap_values[0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
        
        elif type(shap_explainer) == shap.explainers.deep.DeepExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][0][i]
                abs_val = abs(shap_values[0][0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
        
        importances.sort(key=lambda tup: tup[2], reverse = True)
        
        exp.append(importances)

        rel_feat = []

        for i in range(top):
            feat = importances[i]
            if feat[2] > 0:
                rel_feat.append(feat)
                
        rel_exp.append(rel_feat)
        
    return exp, rel_exp

In [8]:
dataset_ref = "bpic2012"
params_dir = PATH + "params"
results_dir = "results"
bucket_method = "single"
cls_encoding = "agg"
cls_method = "xgboost"

gap = 1
n_iter = 1

method_name = "%s_%s"%(bucket_method, cls_encoding)

generate_samples = True
generate_lime = True
generate_kernel_shap = True
generate_model_shap = True

sample_size = 2
exp_iter = 10
max_feat = 10

dataset_ref_to_datasets = {
    #"bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(5,6)],
    "bpic2017" : ["bpic2017_accepted"],
    "bpic2012" : ["bpic2012_accepted"]
    #"insurance": ["insurance_activity", "insurance_followup"],
    #"sepsis_cases": ["sepsis_cases_1", "sepsis_cases_2", "sepsis_cases_4"]
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]

datasets

['bpic2012_accepted']

In [9]:
if generate_samples:

    for dataset_name in datasets:
        
        dataset_manager = DatasetManager(dataset_name)
        
        for ii in range(n_iter):
            num_buckets = range(len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))]))
            
            for bucket in list(num_buckets):
                bucketID = bucket+1
                print ('Bucket', bucketID)

                #import everything needed to sort and predict
                pipeline_path = os.path.join(PATH, "%s/%s_%s/pipelines/pipeline_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                bucketer_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/bucketer_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))

                predictor = joblib.load(pipeline_path)
                cls = joblib.load(cls_path)
                feature_combiner = joblib.load(feat_comb_path)
                bucketer = joblib.load(bucketer_path)

                #import data for bucket
                X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                X_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                Y_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                with open (X_train_path, 'rb') as f:
                    dt_train_bucket = pickle.load(f)
                with open (Y_train_path, 'rb') as f:
                    train_y = pickle.load(f)
                with open (X_test_path, 'rb') as f:
                    dt_test_bucket = pickle.load(f)
                with open (Y_test_path, 'rb') as f:
                    test_y = pickle.load(f)

                #import previous results from predictions
                tn_path = os.path.join(PATH, "%s/%s_%s/instances/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/instances/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/instances/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/instances/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                instances = []

                with open (tn_path, 'rb') as f:
                    tn_list = pickle.load(f)
                with open (tp_path, 'rb') as f:
                    tp_list = pickle.load(f)
                with open (fn_path, 'rb') as f:
                    fn_list = pickle.load(f)
                with open (fp_path, 'rb') as f:
                    fp_list = pickle.load(f)

                #save results to a list
                instances.append(tn_list)
                instances.append(tp_list)
                instances.append(fn_list)
                instances.append(fp_list)

                #choose instances from the event log to explain, based on different prefix lengths
                sample_instances = []
                
                for each in instances:
                    prefixes = set([instance['nr_events'] for instance in each])
                    sample = []
                    for length in prefixes:
                    
                        #Find instances of relevant length
                        relevant = [d for d in each if (d['nr_events'] == length)]
                        #Find instances of different prediction probabilities
                        prs = [0.5, 0.6, 0.7, 0.8, 0.9, 1.1]
                        for i in list(range(len(prs)-1)):
                            low = prs[i]
                            high = prs[i+1]
                            ins = [d for d in relevant if (d['proba'] >= low) & (d['proba'] < high)]
                            if len(ins) >= sample_size:
                                sample.extend(random.sample(ins, k=sample_size))
                            else:
                                sample.extend(ins)
                    sample = sorted(sample, key = lambda i: (i['proba'], i['nr_events']))
                    sample_instances.append(sample)

                tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                with open(tn_path, 'wb') as f:
                    pickle.dump(sample_instances[0], f)
                with open(tp_path, 'wb') as f:
                    pickle.dump(sample_instances[1], f)
                with open(fn_path, 'wb') as f:
                    pickle.dump(sample_instances[2], f)
                with open(fp_path, 'wb') as f:
                    pickle.dump(sample_instances[3], f)

Bucket 1


In [ ]:
if generate_lime:

    for dataset_name in datasets:
        
        dataset_manager = DatasetManager(dataset_name)
        
        for ii in range(n_iter):
            num_buckets = range(len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))]))
            
            for bucket in list(num_buckets):
                bucketID = bucket+1
                print ('Bucket', bucketID)

                #import everything needed to sort and predict
                feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls = joblib.load(cls_path)
                feature_combiner = joblib.load(feat_comb_path)
                
                #import data for bucket
                X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                
                with open (X_train_path, 'rb') as f:
                    dt_train_bucket = pickle.load(f)
                with open (Y_train_path, 'rb') as f:
                    train_y = pickle.load(f)
                #with open (X_test_path, 'rb') as f:
                #    dt_test_bucket = pickle.load(f)
                #with open (Y_test_path, 'rb') as f:
                #    test_y = pickle.load(f)

                #import previously identified samples
                tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                sample_instances = []

                with open (tn_path, 'rb') as f:
                    tn_list = pickle.load(f)
                with open (tp_path, 'rb') as f:
                    tp_list = pickle.load(f)
                with open (fn_path, 'rb') as f:
                    fn_list = pickle.load(f)
                with open (fp_path, 'rb') as f:
                    fp_list = pickle.load(f)

                #save results to a list
                sample_instances.append(tn_list)
                sample_instances.append(tp_list)
                sample_instances.append(fn_list)
                sample_instances.append(fp_list)

                #get the training data as a matrix
                trainingdata = feature_combiner.fit_transform(dt_train_bucket);
                #importance = generate_global_explanations(trainingdata,train_y, cls, feature_combiner)


                #create explainer now that can be passed later
                class_names=['regular','deviant']# regular is 0, deviant is 1, 0 is left, 1 is right
                lime_explainer = lime.lime_tabular.LimeTabularExplainer(trainingdata,
                                              feature_names = feature_combiner.get_feature_names(),
                                              class_names=class_names, discretize_continuous=True)
                
                #explain the chosen instances and find the stability score
                cat_no = 0
                for category in sample_instances:
                    cat_no += 1
                    instance_no = 0
                    for instance in category:
                        instance_no += 1
                        
                        print("Category", cat_no, "of", len(sample_instances), ". Testing", instance_no, "of", len(category), ".")
                        
                        group = instance['input']

                        #print(group.shape,instance['actual'], instance['predicted'])
                        test_x_group= feature_combiner.fit_transform(group) 
                        test_x=np.transpose(test_x_group[0])

                        print('Generating local Explanations for', instance['caseID'])

                        feat_list = feature_combiner.get_feature_names()

                        #Get lime explanations for instance
                        feat_pres = []

                        for iteration in list(range(exp_iter)):
                            print("Run", iteration)
                            lime_exp = generate_lime_explanations(lime_explainer, test_x, cls, instance['actual'])

                            presence_list = [0]*len(feat_list)

                            for each in feat_list:
                                list_idx = feat_list.index(each)
                                #print ("Feature", list_idx)
                                for explanation in lime_exp.as_list():
                                    if each in explanation[0]:
                                        #if lime_exp.as_list().index(explanation) < max_feat:
                                            presence_list[list_idx] = 1

                            feat_pres.append(presence_list)

                        stability = st.getStability(feat_pres)
                        print ("Stability:", round(stability,2))
                        instance['lime_stability'] = stability

                #Save dictionaries updated with stability scores
                with open(tn_path, 'wb') as f:
                    pickle.dump(sample_instances[0], f)
                with open(tp_path, 'wb') as f:
                    pickle.dump(sample_instances[1], f)
                with open(fn_path, 'wb') as f:
                    pickle.dump(sample_instances[2], f)
                with open(fp_path, 'wb') as f:
                    pickle.dump(sample_instances[3], f)

Bucket 1
Category 1 of 4 . Testing 1 of 400 .
Generating local Explanations for 205298_19
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.76
Category 1 of 4 . Testing 2 of 400 .
Generating local Explanations for 210599_4
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Category 1 of 4 . Testing 3 of 400 .
Generating local Explanations for 204730_35
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.76
Category 1 of 4 . Testing 4 of 400 .
Generating local Explanations for 206294_8
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.79
Category 1 of 4 . Testing 5 of 400 .
Generating local Explanations for 209461_2
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.82
Category 1 of 4 . Testing 6 of 400 .
Generating local Explanations for 207717_15
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.8
Category 1 of 4 . Testing 7 of 400 .
Generating local

Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Category 1 of 4 . Testing 54 of 400 .
Generating local Explanations for 206105_33
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.76
Category 1 of 4 . Testing 55 of 400 .
Generating local Explanations for 205465_28
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.79
Category 1 of 4 . Testing 56 of 400 .
Generating local Explanations for 207759_31
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Category 1 of 4 . Testing 57 of 400 .
Generating local Explanations for 207359_37
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Category 1 of 4 . Testing 58 of 400 .
Generating local Explanations for 204472_26
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.8
Category 1 of 4 . Testing 59 of 400 .
Generating local Explanations for 207470_20
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9

Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Category 1 of 4 . Testing 106 of 400 .
Generating local Explanations for 206597_21
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Category 1 of 4 . Testing 107 of 400 .
Generating local Explanations for 206240_30
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.76
Category 1 of 4 . Testing 108 of 400 .
Generating local Explanations for 207467_5
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.74
Category 1 of 4 . Testing 109 of 400 .
Generating local Explanations for 209799_21
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.76
Category 1 of 4 . Testing 110 of 400 .
Generating local Explanations for 205601_39
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.76
Category 1 of 4 . Testing 111 of 400 .
Generating local Explanations for 209910_16
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8

Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Category 1 of 4 . Testing 158 of 400 .
Generating local Explanations for 206132_29
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Category 1 of 4 . Testing 159 of 400 .
Generating local Explanations for 211907_12
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.79
Category 1 of 4 . Testing 160 of 400 .
Generating local Explanations for 207759_30
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Category 1 of 4 . Testing 161 of 400 .
Generating local Explanations for 208136_22
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.76
Category 1 of 4 . Testing 162 of 400 .
Generating local Explanations for 214025_35
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.75
Category 1 of 4 . Testing 163 of 400 .
Generating local Explanations for 206417_12
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 

Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.76
Category 1 of 4 . Testing 210 of 400 .
Generating local Explanations for 205676_25
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.79
Category 1 of 4 . Testing 211 of 400 .
Generating local Explanations for 207831_39
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81
Category 1 of 4 . Testing 212 of 400 .
Generating local Explanations for 207726_16
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Category 1 of 4 . Testing 213 of 400 .
Generating local Explanations for 205727_7
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Category 1 of 4 . Testing 214 of 400 .
Generating local Explanations for 206132_39
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Category 1 of 4 . Testing 215 of 400 .
Generating local Explanations for 210269_8
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8


Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.75
Category 1 of 4 . Testing 262 of 400 .
Generating local Explanations for 209661
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Category 1 of 4 . Testing 263 of 400 .
Generating local Explanations for 207017_24
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.76
Category 1 of 4 . Testing 264 of 400 .
Generating local Explanations for 205849_36
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Category 1 of 4 . Testing 265 of 400 .
Generating local Explanations for 204469_2
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Category 1 of 4 . Testing 266 of 400 .
Generating local Explanations for 209461_5
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Category 1 of 4 . Testing 267 of 400 .
Generating local Explanations for 207969_29
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run

Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Category 1 of 4 . Testing 314 of 400 .
Generating local Explanations for 205459_36
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81
Category 1 of 4 . Testing 315 of 400 .
Generating local Explanations for 206147_37
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Category 1 of 4 . Testing 316 of 400 .
Generating local Explanations for 204499_11
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Category 1 of 4 . Testing 317 of 400 .
Generating local Explanations for 204679_9
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Category 1 of 4 . Testing 318 of 400 .
Generating local Explanations for 208094_4
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Category 1 of 4 . Testing 319 of 400 .
Generating local Explanations for 208031_4
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
R

Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Category 1 of 4 . Testing 366 of 400 .
Generating local Explanations for 208358_22
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Category 1 of 4 . Testing 367 of 400 .
Generating local Explanations for 209910_31
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Category 1 of 4 . Testing 368 of 400 .
Generating local Explanations for 206384_16
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Category 1 of 4 . Testing 369 of 400 .
Generating local Explanations for 205179_12
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81
Category 1 of 4 . Testing 370 of 400 .
Generating local Explanations for 209087_17
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.79
Category 1 of 4 . Testing 371 of 400 .
Generating local Explanations for 209520_5
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8

In [ ]:
if generate_kernel_shap:

    for dataset_name in datasets:
        
        dataset_manager = DatasetManager(dataset_name)
        
        for ii in range(n_iter):
            num_buckets = range(len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))]))
            
            for bucket in list(num_buckets):
                bucketID = bucket+1
                print ('Bucket', bucketID)

                #import everything needed to sort and predict
                feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls = joblib.load(cls_path)
                feature_combiner = joblib.load(feat_comb_path)
                
                #import data for bucket
                X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                
                with open (X_train_path, 'rb') as f:
                    dt_train_bucket = pickle.load(f)
                with open (Y_train_path, 'rb') as f:
                    train_y = pickle.load(f)
                #with open (X_test_path, 'rb') as f:
                #    dt_test_bucket = pickle.load(f)
                #with open (Y_test_path, 'rb') as f:
                #    test_y = pickle.load(f)

                #import previously identified samples
                tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                sample_instances = []

                with open (tn_path, 'rb') as f:
                    tn_list = pickle.load(f)
                with open (tp_path, 'rb') as f:
                    tp_list = pickle.load(f)
                with open (fn_path, 'rb') as f:
                    fn_list = pickle.load(f)
                with open (fp_path, 'rb') as f:
                    fp_list = pickle.load(f)

                #save results to a list
                sample_instances.append(tn_list)
                sample_instances.append(tp_list)
                sample_instances.append(fn_list)
                sample_instances.append(fp_list)

                #create sample of training data to use with Kernel SHAP
                dt_train_sample = pd.DataFrame(columns = dt_train_bucket.columns)
                prefix_lengths = dt_train_bucket['prefix_nr'].unique()
                for length in prefix_lengths:
                    if dt_train_bucket[dt_train_bucket['prefix_nr']==length].shape[0] > 1000:
                        sample = dt_train_bucket[dt_train_bucket['prefix_nr']==length].sample(1000)
                        dt_train_sample = dt_train_sample.append(sample)
                    else:
                        dt_train_sample = dt_train_sample.append(dt_train_bucket[dt_train_bucket['event_nr']==length])
                
                #get the training data sample as a matrix
                trainingsample = feature_combiner.fit_transform(dt_train_sample)
                #importance = generate_global_explanations(trainingdata,train_y, cls, feature_combiner)


                #create explainers now that can be passed later
                #data_sample = shap.kmeans(trainingdata)#, see "KMEANS EXPERIMENT" cell
                shap_explainer = shap.KernelExplainer(cls.predict, trainingsample)

                #explain the chosen instances and find the stability score
                cat_no = 0
                for category in sample_instances:
                    cat_no += 1
                    instance_no = 0
                    
                    for instance in category:
                        instance_no += 1    
                        print("Category", cat_no, "of", len(sample_instances), ". Testing", instance_no, "of", len(category), ".")
                
                        group = instance['input']

                        print(group.shape,instance['actual'], instance['predicted'])
                        test_x_group= feature_combiner.fit_transform(group) 
                        test_x=np.transpose(test_x_group[0])

                        print('Generating local Explanations for', instance['caseID'])

                        feat_list = feature_combiner.get_feature_names()

                        #Get Kernel SHAP explanations for instance
                        exp, rel_exp = create_samples(shap_explainer, exp_iter, test_x.reshape(-1, len(test_x)), feat_list, top = max_feat)

                        feat_pres = []

                        for iteration in rel_exp:
                            print("Kernel SHAP Iteration", rel_exp.index(iteration))
                            #print(iteration)

                            #Stability by index
                            presence_list = [0]*len(feat_list)

                            for each in feat_list:
                                list_idx = feat_list.index(each)
                                #print ("Feature", list_idx)
                                for explanation in iteration:
                                    if each in explanation[0]:
                                        #by index
                                        presence_list[list_idx] = 1

                            feat_pres.append(presence_list)

                        stability = st.getStability(feat_pres)
                        print ("Stability:", round(stability,2))
                        instance['kernel_shap_stability'] = stability
                        
                with open(tn_path, 'wb') as f:
                    pickle.dump(sample_instances[0], f)
                with open(tp_path, 'wb') as f:
                    pickle.dump(sample_instances[1], f)
                with open(fn_path, 'wb') as f:
                    pickle.dump(sample_instances[2], f)
                with open(fp_path, 'wb') as f:
                    pickle.dump(sample_instances[3], f)

In [ ]:
if generate_model_shap:

    for dataset_name in datasets:
        
        dataset_manager = DatasetManager(dataset_name)
        
        for ii in range(n_iter):
            num_buckets = range(len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))]))
            
            for bucket in list(num_buckets):
                bucketID = bucket+1
                print ('Bucket', bucketID)

                #import everything needed to sort and predict
                feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls = joblib.load(cls_path)
                feature_combiner = joblib.load(feat_comb_path)
                
                #import data for bucket
                #X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                #Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                
                #with open (X_train_path, 'rb') as f:
                #    dt_train_bucket = pickle.load(f)
                #with open (Y_train_path, 'rb') as f:
                #    train_y = pickle.load(f)
                #with open (X_test_path, 'rb') as f:
                #    dt_test_bucket = pickle.load(f)
                #with open (Y_test_path, 'rb') as f:
                #    test_y = pickle.load(f)

                #import previously identified samples
                tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                sample_instances = []

                with open (tn_path, 'rb') as f:
                    tn_list = pickle.load(f)
                with open (tp_path, 'rb') as f:
                    tp_list = pickle.load(f)
                with open (fn_path, 'rb') as f:
                    fn_list = pickle.load(f)
                with open (fp_path, 'rb') as f:
                    fp_list = pickle.load(f)

                #save results to a list
                sample_instances.append(tn_list)
                sample_instances.append(tp_list)
                sample_instances.append(fn_list)
                sample_instances.append(fp_list)

               #create explainers now that can be passed later
               tree_explainer = shap.TreeExplainer(cls)

                #explain the chosen instances and find the stability score
                cat_no = 0
                for category in sample_instances:
                    cat_no += 1
                    instance_no = 0
                    
                    for instance in category:
                        instance_no += 1    
                        print("Category", cat_no, "of", len(sample_instances), ". Testing", instance_no, "of", len(category), ".")
                
                        group = instance['input']

                        print(group.shape,instance['actual'], instance['predicted'])
                        test_x_group= feature_combiner.fit_transform(group) 
                        test_x=np.transpose(test_x_group[0])

                        print('Generating local Explanations for', instance['caseID'])

                        feat_list = feature_combiner.get_feature_names()

                        #Get Tree SHAP explanations for instance
                        exp, rel_exp = create_samples(tree_explainer, exp_iter, test_x.reshape(-1, len(test_x)), feat_list, top = max_feat)

                        feat_pres = []

                        for iteration in rel_exp:
                            print("Tree SHAP Iteration", rel_exp.index(iteration))
                            #print(iteration)

                            #Stability by index
                            presence_list = [0]*len(feat_list)

                            for each in feat_list:
                                list_idx = feat_list.index(each)
                                #print ("Feature", list_idx)
                                for explanation in iteration:
                                    if each in explanation[0]:
                                        #by index
                                        presence_list[list_idx] = 1

                            feat_pres.append(presence_list)

                        stability = st.getStability(feat_pres)
                        print ("Stability:", round(stability,2))
                        instance['tree_shap_stability'] = stability

                with open(tn_path, 'wb') as f:
                    pickle.dump(sample_instances[0], f)
                with open(tp_path, 'wb') as f:
                    pickle.dump(sample_instances[1], f)
                with open(fn_path, 'wb') as f:
                    pickle.dump(sample_instances[2], f)
                with open(fp_path, 'wb') as f:
                    pickle.dump(sample_instances[3], f)